<a href="https://colab.research.google.com/github/Sidhtang/rr-digital-project-/blob/main/kyc_form_application.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

import datetime
import pandas as pd
from PIL import Image
import json

def initialize_session_state():
    """Initialize session state variables if they don't exist"""
    if 'step' not in st.session_state:
        st.session_state.step = 1
    if 'form_data' not in st.session_state:
        st.session_state.form_data = {
            'personal_info': {},
            'contact_info': {},
            'document_info': {}
        }

def render_progress_bar():
    """Render progress bar based on current step"""
    progress = (st.session_state.step - 1) / 3
    st.progress(progress)
    steps = ['Personal Information', 'Contact Details', 'Document Verification', 'Review']
    st.write(f"Step {st.session_state.step} of 4: {steps[st.session_state.step-1]}")

def personal_info_step():
    """Render personal information form fields"""
    with st.form("personal_info_form"):
        given_name = st.text_input("Given Name",
                                 value=st.session_state.form_data['personal_info'].get('given_name', ''))
        family_name = st.text_input("Family Name",
                                  value=st.session_state.form_data['personal_info'].get('family_name', ''))
        date_of_birth = st.date_input("Date of Birth",
                                    value=datetime.datetime.strptime(st.session_state.form_data['personal_info'].get('date_of_birth', '2000-01-01'), '%Y-%m-%d').date())
        tax_id = st.text_input("Tax ID (SSN)",
                             value=st.session_state.form_data['personal_info'].get('tax_id', ''))

        submitted = st.form_submit_button("Next")
        if submitted and given_name and family_name and tax_id:
            st.session_state.form_data['personal_info'] = {
                'given_name': given_name,
                'family_name': family_name,
                'date_of_birth': date_of_birth.strftime('%Y-%m-%d'),
                'tax_id': tax_id
            }
            st.session_state.step += 1
            st.experimental_rerun()
        elif submitted:
            st.error("Please fill in all required fields")

def contact_info_step():
    """Render contact information form fields"""
    with st.form("contact_info_form"):
        street_address = st.text_input("Street Address",
                                     value=st.session_state.form_data['contact_info'].get('street_address', ''))
        city = st.text_input("City",
                           value=st.session_state.form_data['contact_info'].get('city', ''))
        state = st.selectbox("State",
                           options=['', 'NY', 'CA', 'TX', 'FL', 'IL'],
                           index=['', 'NY', 'CA', 'TX', 'FL', 'IL'].index(st.session_state.form_data['contact_info'].get('state', '')))
        postal_code = st.text_input("Postal Code",
                                  value=st.session_state.form_data['contact_info'].get('postal_code', ''))

        col1, col2 = st.columns(2)
        with col1:
            back = st.form_submit_button("Back")
        with col2:
            submitted = st.form_submit_button("Next")

        if back:
            st.session_state.step -= 1
            st.experimental_rerun()
        elif submitted and street_address and city and state and postal_code:
            st.session_state.form_data['contact_info'] = {
                'street_address': street_address,
                'city': city,
                'state': state,
                'postal_code': postal_code
            }
            st.session_state.step += 1
            st.experimental_rerun()
        elif submitted:
            st.error("Please fill in all required fields")

def document_info_step():
    """Render document verification form fields"""
    with st.form("document_info_form"):
        document_type = st.selectbox("Document Type",
                                   options=['', 'Passport', 'Driver License', 'National ID'],
                                   index=['', 'Passport', 'Driver License', 'National ID'].index(
                                       st.session_state.form_data['document_info'].get('document_type', '')))
        document_number = st.text_input("Document Number",
                                      value=st.session_state.form_data['document_info'].get('document_number', ''))

        uploaded_file = st.file_uploader("Upload Document", type=['pdf', 'png', 'jpg', 'jpeg'])
        if uploaded_file is not None:
            st.image(uploaded_file, caption='Uploaded Document', width=300)

        col1, col2 = st.columns(2)
        with col1:
            back = st.form_submit_button("Back")
        with col2:
            submitted = st.form_submit_button("Next")

        if back:
            st.session_state.step -= 1
            st.experimental_rerun()
        elif submitted and document_type and document_number:
            st.session_state.form_data['document_info'] = {
                'document_type': document_type,
                'document_number': document_number,
                'document_uploaded': uploaded_file is not None
            }
            st.session_state.step += 1
            st.experimental_rerun()
        elif submitted:
            st.error("Please fill in all required fields")

def review_step():
    """Render review page with all collected information"""
    st.write("### Review Your Information")

    st.write("#### Personal Information")
    for key, value in st.session_state.form_data['personal_info'].items():
        st.write(f"**{key.replace('_', ' ').title()}:** {value}")

    st.write("#### Contact Information")
    for key, value in st.session_state.form_data['contact_info'].items():
        st.write(f"**{key.replace('_', ' ').title()}:** {value}")

    st.write("#### Document Information")
    for key, value in st.session_state.form_data['document_info'].items():
        st.write(f"**{key.replace('_', ' ').title()}:** {value}")

    col1, col2 = st.columns(2)
    with col1:
        if st.button("Back"):
            st.session_state.step -= 1
            st.experimental_rerun()
    with col2:
        if st.button("Submit"):
            # Here you would typically send the data to your backend
            st.success("Form submitted successfully!")
            st.json(st.session_state.form_data)

def main():
    st.title("KYC Form")

    # Initialize session state
    initialize_session_state()

    # Render progress bar
    render_progress_bar()

    # Render appropriate step
    if st.session_state.step == 1:
        personal_info_step()
    elif st.session_state.step == 2:
        contact_info_step()
    elif st.session_state.step == 3:
        document_info_step()
    elif st.session_state.step == 4:
        review_step()

if __name__ == "__main__":
    main()

2024-11-23 05:41:35.120 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:41:35.494 
  command:

    streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2024-11-23 05:41:35.499 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:41:35.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:41:35.507 Session state does not function when running a script without `streamlit run`
2024-11-23 05:41:35.512 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:41:35.514 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-11-23 05:41:35.521 Thread 'MainThread': missing ScriptRunContext! 

In [ ]:
! streamlit run /usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py




  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://104.196.215.190:8501

  Stopping...
  Stopping...


In [ ]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 14.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 19.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 71.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
import gradio as gr
import datetime
import json
from PIL import Image
import numpy as np

class KYCForm:
    def __init__(self):
        self.form_data = {
            'personal_info': {},
            'contact_info': {},
            'document_info': {}
        }
        self.current_step = 1
        self.states = ['AL', 'AK', 'AZ', 'AR', 'CA', 'CO', 'CT', 'DE', 'FL', 'GA', 'HI', 'ID', 'IL', 'IN', 'IA', 'KS', 'KY', 'LA', 'ME', 'MD', 'MA', 'MI', 'MN', 'MS', 'MO', 'MT', 'NE', 'NV', 'NH', 'NJ', 'NM', 'NY', 'NC', 'ND', 'OH', 'OK', 'OR', 'PA', 'RI', 'SC', 'SD', 'TN', 'TX', 'UT', 'VT', 'VA', 'WA', 'WV', 'WI', 'WY']
        self.document_types = ['Passport', 'Driver License', 'National ID']

    def get_progress(self):
        return f"Step {self.current_step} of 4"

    def validate_date(self, date_str):
        try:
            # Try to parse the date string (YYYY-MM-DD format)
            datetime.datetime.strptime(date_str, '%Y-%m-%d')
            return True
        except ValueError:
            return False

    def update_personal_info(self, given_name, family_name, date_of_birth, tax_id):
        if not all([given_name, family_name, date_of_birth, tax_id]):
            return {
                gr.update(value=self.current_step),  # step
                gr.update(value="Please fill in all required fields"),  # error
                None,  # next_tab
                gr.update(visible=True),  # error_visible
                self.get_progress()  # progress
            }

        if not self.validate_date(date_of_birth):
            return {
                gr.update(value=self.current_step),
                gr.update(value="Please enter a valid date in YYYY-MM-DD format"),
                None,
                gr.update(visible=True),
                self.get_progress()
            }

        self.form_data['personal_info'] = {
            'given_name': given_name,
            'family_name': family_name,
            'date_of_birth': date_of_birth,
            'tax_id': tax_id
        }
        self.current_step = 2
        return {
            gr.update(value=self.current_step),  # step
            gr.update(value=""),  # error
            gr.update(selected="contact_tab"),  # next_tab
            gr.update(visible=False),  # error_visible
            self.get_progress()  # progress
        }

    def update_contact_info(self, street_address, city, state, postal_code):
        if not all([street_address, city, state, postal_code]):
            return {
                gr.update(value=self.current_step),
                gr.update(value="Please fill in all required fields"),
                None,
                gr.update(visible=True),
                self.get_progress()
            }

        self.form_data['contact_info'] = {
            'street_address': street_address,
            'city': city,
            'state': state,
            'postal_code': postal_code
        }
        self.current_step = 3
        return {
            gr.update(value=self.current_step),
            gr.update(value=""),
            gr.update(selected="document_tab"),
            gr.update(visible=False),
            self.get_progress()
        }

    def update_document_info(self, document_type, document_number, document_file):
        if not all([document_type, document_number, document_file is not None]):
            return {
                gr.update(value=self.current_step),
                gr.update(value="Please fill in all required fields and upload a document"),
                None,
                gr.update(visible=True),
                self.get_progress()
            }

        self.form_data['document_info'] = {
            'document_type': document_type,
            'document_number': document_number,
            'document_uploaded': True
        }
        self.current_step = 4
        return {
            gr.update(value=self.current_step),
            gr.update(value=""),
            gr.update(selected="review_tab"),
            gr.update(visible=False),
            self.get_progress()
        }

    def go_back(self, current_tab):
        tab_mapping = {
            "contact_tab": ("personal_tab", 1),
            "document_tab": ("contact_tab", 2),
            "review_tab": ("document_tab", 3)
        }

        if current_tab in tab_mapping:
            prev_tab, step = tab_mapping[current_tab]
            self.current_step = step
            return {
                gr.update(value=step),
                gr.update(value=""),
                gr.update(selected=prev_tab),
                gr.update(visible=False),
                self.get_progress()
            }
        return None

    def submit_form(self):
        return json.dumps(self.form_data, indent=2)

def create_interface():
    kyc = KYCForm()

    with gr.Blocks(css="""
        .gradio-container { max-width: 800px; margin: auto; }
        .error-message { color: red; }
        .helper-text { font-size: 0.8em; color: #666; }
        """) as interface:
        gr.Markdown("# KYC Form Application")

        # Hidden state trackers
        step = gr.Number(value=1, visible=False)
        error = gr.Textbox(visible=False)
        error_visible = gr.Checkbox(visible=False, value=False)

        # Progress indicator
        progress = gr.Markdown(value=kyc.get_progress())

        with gr.Row():
            error_box = gr.Markdown(visible=False, elem_classes=["error-message"])

        with gr.Tabs() as tabs:
            # Personal Information Tab
            with gr.Tab("Personal Information", id="personal_tab"):
                given_name = gr.Textbox(label="Given Name*", placeholder="Enter your given name")
                family_name = gr.Textbox(label="Family Name*", placeholder="Enter your family name")
                date_of_birth = gr.Textbox(
                    label="Date of Birth* (YYYY-MM-DD)",
                    placeholder="e.g., 1990-01-31"
                )
                gr.Markdown("*Please enter date in YYYY-MM-DD format", elem_classes=["helper-text"])
                tax_id = gr.Textbox(label="Tax ID (SSN)*", placeholder="Enter your SSN")
                personal_next = gr.Button("Next")

            # Contact Information Tab
            with gr.Tab("Contact Information", id="contact_tab"):
                street_address = gr.Textbox(label="Street Address*", placeholder="Enter your street address")
                city = gr.Textbox(label="City*", placeholder="Enter your city")
                state = gr.Dropdown(label="State*", choices=kyc.states)
                postal_code = gr.Textbox(label="Postal Code*", placeholder="Enter your postal code")
                with gr.Row():
                    contact_back = gr.Button("Back")
                    contact_next = gr.Button("Next")

            # Document Information Tab
            with gr.Tab("Document Verification", id="document_tab"):
                document_type = gr.Dropdown(label="Document Type*", choices=kyc.document_types)
                document_number = gr.Textbox(label="Document Number*", placeholder="Enter document number")
                document_file = gr.File(label="Upload Document*", file_types=["image", "pdf"])
                with gr.Row():
                    document_back = gr.Button("Back")
                    document_next = gr.Button("Next")

            # Review Tab
            with gr.Tab("Review", id="review_tab"):
                review_text = gr.JSON(label="Review your information")
                with gr.Row():
                    review_back = gr.Button("Back")
                    submit = gr.Button("Submit", variant="primary")

        # Event handlers
        personal_next.click(
            kyc.update_personal_info,
            inputs=[given_name, family_name, date_of_birth, tax_id],
            outputs=[step, error, tabs, error_visible, progress]
        ).then(
            lambda x: x,
            inputs=[error],
            outputs=[error_box]
        )

        contact_next.click(
            kyc.update_contact_info,
            inputs=[street_address, city, state, postal_code],
            outputs=[step, error, tabs, error_visible, progress]
        ).then(
            lambda x: x,
            inputs=[error],
            outputs=[error_box]
        )

        document_next.click(
            kyc.update_document_info,
            inputs=[document_type, document_number, document_file],
            outputs=[step, error, tabs, error_visible, progress]
        ).then(
            lambda x: x,
            inputs=[error],
            outputs=[error_box]
        )

        contact_back.click(
            lambda: kyc.go_back("contact_tab"),
            outputs=[step, error, tabs, error_visible, progress]
        )

        document_back.click(
            lambda: kyc.go_back("document_tab"),
            outputs=[step, error, tabs, error_visible, progress]
        )

        review_back.click(
            lambda: kyc.go_back("review_tab"),
            outputs=[step, error, tabs, error_visible, progress]
        )

        submit.click(
            kyc.submit_form,
            outputs=review_text
        )

    return interface

if __name__ == "__main__":
    interface = create_interface()
    interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6c0739aac47465aa03.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
